# N-gram Language Models - Practice with Real Data

**Course:** Natural Language Processing  
**Topic:** Building N-gram Models with HuggingFace Datasets  

In this notebook, you will:
1. Load real text corpora from HuggingFace
2. Implement n-gram language models from scratch
3. Train and evaluate your models
4. Generate text and compare different approaches

## Setup

In [1]:
# Install required packages
!pip install datasets -q

import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
import random
from datasets import load_dataset
from tqdm import tqdm

# Set random seed
np.random.seed(42)
random.seed(42)

print("✓ Setup complete!")

✓ Setup complete!


## Part 1: Load Dataset from HuggingFace

We'll use the **WikiText-2** dataset, which contains good quality text from Wikipedia articles.

In [2]:
# Load WikiText-2 dataset
print("Loading WikiText-2 dataset...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

print("\nDataset structure:")
print(dataset)

# Explore the data
print("\nSample from training set:")
print(dataset['train'][0])
print("\n" + "="*60)
print(dataset['train'][1])

Loading WikiText-2 dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]


Dataset structure:
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

Sample from training set:
{'text': ''}

{'text': ' = Valkyria Chronicles III = \n'}


### Exercise 1.1: Data Exploration

Answer these questions:
1. How many examples are in train/validation/test splits?
2. What does each example look like?
3. How many empty or very short texts are there?

{'text': ['', ' = Valkyria Chronicles III = \n', '', ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game m

In [7]:
# Question 1: Count examples in each split
train_size = len(dataset['train'])
valid_size = len(dataset['validation'])
test_size = len(dataset['test'])

print(f"Train size: {train_size}")
print(f"Validation size: {valid_size}")
print(f"Test size: {test_size}")

# Question 2: Look at a few examples
print("\nFirst 5 examples from training set:")
# TODO: YOUR CODE HERE - print first 5 examples
print(dataset['train'][:5])


# Question 3: Check for empty/short texts
print("\nChecking for empty or short texts:")
# TODO: YOUR CODE HERE - count texts with length 0 or < 5 words
# Hint: Use len(text['text'].split())
empty_count = sum(1 for text in dataset['train'] if len(text['text'].split()) == 0)
short_count = sum(1 for text in dataset['train'] if len(text['text'].split()) < 5)
print(f"Number of empty texts: {empty_count}")
print(f"Number of short texts (<5 words): {short_count}")

Train size: 36718
Validation size: 3760
Test size: 4358

First 5 examples from training set:
{'text': ['', ' = Valkyria Chronicles III = \n', '', ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standar

## Part 2: Data Preprocessing

### Exercise 2.1: Implement Text Preprocessing

In [9]:
def preprocess_text(text: str, add_start_end: bool = True) -> List[str]:
    """
    Preprocess text:
    - Convert to lowercase
    - Split into tokens (simple whitespace tokenization)
    - Add <s> and </s> tokens if requested

    Args:
        text: Input text
        add_start_end: Whether to add start/end tokens

    Returns:
        List of tokens
    """
    # TODO: Implement this function
    # Step 1: Convert to lowercase
    text= text.lower()

    # Step 2: Split on whitespace
    tokens = text.split()

    # Step 3: Add <s> and </s> if requested
    if add_start_end:
        tokens = ['<s>'] + tokens + ['</s>']
        return tokens

    pass


# Test your function
test_text = "Hello World! This is a test."
tokens = preprocess_text(test_text, add_start_end=True)
print(f"Input: {test_text}")
print(f"Expected: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']")
print(f"Your output: {tokens}")

Input: Hello World! This is a test.
Expected: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']
Your output: ['<s>', 'hello', 'world!', 'this', 'is', 'a', 'test.', '</s>']


### Exercise 2.2: Prepare Training Corpus

In [12]:
def prepare_corpus(dataset_split, max_examples: int = None) -> List[List[str]]:
    """
    Prepare corpus from dataset split.

    Args:
        dataset_split: HuggingFace dataset split
        max_examples: Maximum number of examples to use (None = all)

    Returns:
        List of tokenized sentences
    """
    corpus = []

    # TODO: Implement this function
    # 1. Iterate through the dataset
    # 2. Skip empty texts (text['text'].strip() == '')
    # 3. Preprocess each text using your function above
    # 4. Add to corpus only if result has more than 2 tokens
    # 5. Stop at max_examples if specified

    # Hint: Use this structure:
    # count = 0
    # for item in tqdm(dataset_split, desc="Processing"):
    #     text = item['text'].strip()
    #     if not text:
    #         continue
    #     ...
    count = 0
    for item in tqdm(dataset_split, desc="Processing"):
        text = item['text'].strip()
        if not text:
            continue
        tokens = preprocess_text(text, add_start_end=True)
        if len(tokens) > 2:
            corpus.append(tokens)
            count += 1
        if max_examples and count >= max_examples:
            break
    return corpus

    pass

# Prepare training corpus (use subset for speed)
print("Preparing training corpus...")
train_corpus = prepare_corpus(dataset['train'], max_examples=1000)

print(f"\nCorpus size: {len(train_corpus)} sentences")
print(f"First sentence: {train_corpus[0][:10]}...")  # Show first 10 tokens

Preparing training corpus...


Processing:   4%|▍         | 1538/36718 [00:00<00:01, 17777.72it/s]


Corpus size: 1000 sentences
First sentence: ['<s>', '=', 'valkyria', 'chronicles', 'iii', '=', '</s>']...


## Part 3: Build Vocabulary

### Exercise 3.1: Build Vocabulary

In [13]:
def build_vocabulary(corpus: List[List[str]]) -> set:
    """
    Build vocabulary from corpus.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Set of unique tokens
    """
    # TODO: Implement this function
    # Hint: Create a set and add all tokens from all sentences
    vocab = set()
    for sentence in corpus:
        for token in sentence:
            vocab.add(token)
    return vocab

    pass

# Build vocabulary
vocab = build_vocabulary(train_corpus)
print(f"Vocabulary size: {len(vocab)}")
print(f"Sample words: {list(vocab)[:20]}")

Vocabulary size: 10690
Sample words: ['score', 'seller', 'q.', 'maat', 'calibers', 'symbolize', 'gigs', 'intervals', 'tertiary', 'andrews', 'qualified', 'emotional', 'constrained', 'way', 'decked', 'insurance', 'parliament', 'engage', 'trial', 'marines']


## Part 4: Implement N-gram Counting

### Exercise 4.1: Count Unigrams

In [15]:
def count_unigrams(corpus: List[List[str]]) -> Counter:
    """
    Count unigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with unigram counts
    """
    unigram_counts = Counter()

    # TODO: Implement this function
    # Iterate through corpus and count each token
    for sentence in corpus:
        for token in sentence:
            unigram_counts[token] += 1
    return unigram_counts

    pass

# Test
unigram_counts = count_unigrams(train_corpus)
print(f"Total unigrams: {sum(unigram_counts.values())}")
print(f"Unique unigrams: {len(unigram_counts)}")
print(f"\nMost common unigrams:")
print(unigram_counts.most_common(20))

Total unigrams: 90023
Unique unigrams: 10690

Most common unigrams:
[('the', 5757), (',', 4468), ('.', 3203), ('of', 2568), ('and', 2237), ('in', 1927), ('to', 1655), ('a', 1574), ('=', 1191), ('<s>', 1000), ('</s>', 1000), ('"', 961), ('was', 865), ('@-@', 782), ('with', 697), ('as', 663), ('for', 662), ('that', 623), ("'s", 606), ('on', 594)]


### Exercise 4.2: Count Bigrams

In [16]:
def count_bigrams(corpus: List[List[str]]) -> Counter:
    """
    Count bigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with bigram counts (bigrams as tuples)
    """
    bigram_counts = Counter()

    # TODO: Implement this function
    # For each sentence, create pairs of consecutive words
    # Store as tuples: (word1, word2)
    for sentence in corpus:
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            bigram_counts[bigram] += 1
    return bigram_counts

    pass

# Test
bigram_counts = count_bigrams(train_corpus)
print(f"Total bigrams: {sum(bigram_counts.values())}")
print(f"Unique bigrams: {len(bigram_counts)}")
print(f"\nMost common bigrams:")
print(bigram_counts.most_common(20))

Total bigrams: 89023
Unique bigrams: 50573

Most common bigrams:
[(('of', 'the'), 837), (('=', '='), 656), ((',', 'and'), 607), (('.', '</s>'), 578), (('.', 'the'), 526), (('in', 'the'), 498), ((',', 'the'), 316), (('to', 'the'), 268), (('<s>', '='), 264), (('=', '</s>'), 264), (('on', 'the'), 199), (('<s>', 'the'), 181), (('for', 'the'), 178), (('and', 'the'), 177), (('with', 'the'), 170), (('at', 'the'), 157), (('.', 'in'), 156), (('.', '"'), 130), ((',', 'a'), 127), ((',', 'which'), 124)]


### Exercise 4.3: Count Trigrams

In [17]:
def count_trigrams(corpus: List[List[str]]) -> Counter:
    """
    Count trigram frequencies.

    Args:
        corpus: List of tokenized sentences

    Returns:
        Counter with trigram counts (trigrams as tuples)
    """
    trigram_counts = Counter()

    # TODO: Implement this function
    # For each sentence, create triples of consecutive words
    # Store as tuples: (word1, word2, word3)
    for sentence in corpus:
        for i in range(len(sentence) - 2):
            trigram = (sentence[i], sentence[i + 1], sentence[i + 2])
            trigram_counts[trigram] += 1
    return trigram_counts

    pass

# Test
trigram_counts = count_trigrams(train_corpus)
print(f"Total trigrams: {sum(trigram_counts.values())}")
print(f"Unique trigrams: {len(trigram_counts)}")
print(f"\nMost common trigrams:")
print(trigram_counts.most_common(20))

Total trigrams: 88023
Unique trigrams: 76398

Most common trigrams:
[(('<s>', '=', '='), 230), (('=', '=', '</s>'), 230), (('=', '=', '='), 196), ((',', 'and', 'the'), 65), (('the', 'blue', 'jackets'), 39), (('one', 'of', 'the'), 37), (('.', '"', '</s>'), 34), ((';', 'blackie', ','), 30), (('as', 'well', 'as'), 29), (('"', '.', '</s>'), 27), ((',', 'such', 'as'), 27), (('the', 'gold', 'dollar'), 27), (('strapping', 'young', 'lad'), 26), (('the', 'united', 'states'), 25), (('.', 'she', 'was'), 24), (('.', 'it', 'was'), 22), ((',', 'and', 'a'), 22), (('.', 'in', 'the'), 22), (('south', 'of', 'heaven'), 22), (('in', 'the', 'uk'), 21)]


## Part 5: Implement Language Models

### Exercise 5.1: Unigram Model

In [ ]:
class UnigramModel:
    """Unigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.total_words = sum(self.unigram_counts.values())

    def probability(self, word: str) -> float:
        """Calculate P(word)."""
        # TODO: Implement this
        # P(word) = C(word) / N
        pass

    def log_probability(self, word: str) -> float:
        """Calculate log P(word)."""
        # TODO: Implement this
        # Handle zero probability by returning float('-inf')
        pass

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # TODO: Implement this
        # Sum of log probabilities of all words
        pass

# Test your model
unigram_model = UnigramModel(train_corpus)
test_words = ["the", "of", "and", "<s>", "</s>"]
print("Unigram Probabilities:")
for word in test_words:
    prob = unigram_model.probability(word)
    print(f"  P({word}) = {prob:.6f}")

### Exercise 5.2: Bigram Model

In [ ]:
class BigramModel:
    """Bigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)

    def probability(self, word: str, previous_word: str) -> float:
        """Calculate P(word | previous_word)."""
        # TODO: Implement this
        # P(wi | wi-1) = C(wi-1, wi) / C(wi-1)
        pass

    def log_probability(self, word: str, previous_word: str) -> float:
        """Calculate log P(word | previous_word)."""
        # TODO: Implement this
        pass

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # TODO: Implement this
        # Sum log probabilities: log P(w2|w1) + log P(w3|w2) + ...
        pass

# Test your model
bigram_model = BigramModel(train_corpus)
test_bigrams = [("<s>", "the"), ("the", "and"), ("of", "the")]
print("Bigram Probabilities:")
for w1, w2 in test_bigrams:
    prob = bigram_model.probability(w2, w1)
    print(f"  P({w2} | {w1}) = {prob:.6f}")

### Exercise 5.3: Trigram Model

In [ ]:
class TrigramModel:
    """Trigram language model."""

    def __init__(self, corpus: List[List[str]]):
        self.bigram_counts = count_bigrams(corpus)
        self.trigram_counts = count_trigrams(corpus)

    def probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate P(word | prev_word2, prev_word1)."""
        # TODO: Implement this
        # P(wi | wi-2, wi-1) = C(wi-2, wi-1, wi) / C(wi-2, wi-1)
        pass

    def log_probability(self, word: str, prev_word1: str, prev_word2: str) -> float:
        """Calculate log P(word | prev_word2, prev_word1)."""
        # TODO: Implement this
        pass

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # TODO: Implement this
        pass

# Test your model
trigram_model = TrigramModel(train_corpus)
if len(train_corpus[0]) >= 3:
    w1, w2, w3 = train_corpus[0][:3]
    prob = trigram_model.probability(w3, w2, w1)
    print(f"P({w3} | {w1}, {w2}) = {prob:.6f}")

## Part 6: Model Evaluation - Perplexity

### Exercise 6.1: Implement Perplexity Calculation

In [ ]:
def calculate_perplexity(model, test_corpus: List[List[str]], model_type: str = "bigram") -> float:
    """
    Calculate perplexity for a language model.

    Perplexity = exp(-1/N * sum(log P(wi | context)))

    Args:
        model: Language model with sentence_log_probability method
        test_corpus: List of test sentences
        model_type: 'unigram', 'bigram', or 'trigram'

    Returns:
        Perplexity score
    """
    # TODO: Implement this function
    # 1. Calculate total log probability across all sentences
    # 2. Count total words (excluding <s> for bigram/trigram)
    # 3. Calculate average log probability
    # 4. Return exp(-avg_log_prob)

    pass

# Prepare test corpus
print("Preparing test corpus...")
test_corpus = prepare_corpus(dataset['validation'], max_examples=100)
print(f"Test corpus size: {len(test_corpus)} sentences")

# Calculate perplexities
print("\nCalculating perplexities...")
# TODO: Calculate and print perplexities for all three models

## Part 7: Text Generation

### Exercise 7.1: Implement Text Generator

In [ ]:
class BigramGenerator:
    """Generate text using bigram model."""

    def __init__(self, corpus: List[List[str]]):
        self.bigram_model = BigramModel(corpus)
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)
        self.next_word_probs = self._build_next_word_distribution()

    def _build_next_word_distribution(self) -> Dict:
        """Build distribution over next words for each word."""
        # TODO: Implement this
        # For each word w1, build dict of {w2: P(w2|w1)}
        pass

    def generate(self, max_length: int = 20) -> List[str]:
        """Generate a sentence."""
        # TODO: Implement this
        # 1. Start with '<s>'
        # 2. Sample next word based on probabilities
        # 3. Stop at '</s>' or max_length
        # Hint: Use np.random.choice(words, p=probs)
        pass

# Test generator
generator = BigramGenerator(train_corpus)
print("Generated Sentences:")
print("=" * 60)
for i in range(10):
    sentence = generator.generate(max_length=20)
    print(f"{i+1}. {' '.join(sentence)}")

## Part 8: Smoothing

### Exercise 8.1: Implement Add-One Smoothing

In [ ]:
class BigramModelSmoothed:
    """Bigram model with add-one smoothing."""

    def __init__(self, corpus: List[List[str]]):
        self.unigram_counts = count_unigrams(corpus)
        self.bigram_counts = count_bigrams(corpus)
        self.vocab = build_vocabulary(corpus)
        self.vocab_size = len(self.vocab)

    def probability(self, word: str, previous_word: str) -> float:
        """Calculate P(word | previous_word) with add-one smoothing.

        Formula: P(wi | wi-1) = (C(wi-1, wi) + 1) / (C(wi-1) + V)
        """
        # TODO: Implement this
        pass

    def log_probability(self, word: str, previous_word: str) -> float:
        """Calculate log P(word | previous_word)."""
        # TODO: Implement this
        pass

    def sentence_log_probability(self, sentence: List[str]) -> float:
        """Calculate log P(sentence)."""
        # TODO: Implement this
        pass

# Compare with unsmoothed model
smoothed_model = BigramModelSmoothed(train_corpus)

# Test on unseen bigrams
test_unseen = [("<s>", "quantum"), ("the", "xylophone"), ("amazing", "unicorn")]
print("Comparison: Unsmoothed vs Smoothed")
print("=" * 60)
for w1, w2 in test_unseen:
    unsmoothed = bigram_model.probability(w2, w1)
    smoothed = smoothed_model.probability(w2, w1)
    print(f"P({w2} | {w1}):")
    print(f"  Unsmoothed: {unsmoothed:.8f}")
    print(f"  Smoothed:   {smoothed:.8f}")
    print()

## Part 9: Analysis and Comparison

### Exercise 9.1: Compare All Models

In [ ]:
# TODO: Create a comparison table showing:
# 1. Number of parameters (unique n-grams)
# 2. Training perplexity
# 3. Test perplexity
# 4. Sample generated sentences

import pandas as pd

comparison_data = {
    'Model': ['Unigram', 'Bigram', 'Trigram', 'Bigram (Smoothed)'],
    'Parameters': [
        len(unigram_counts),
        len(bigram_counts),
        len(trigram_counts),
        len(bigram_counts)
    ],
    'Train Perplexity': [
        # TODO: Fill in
        0.0, 0.0, 0.0, 0.0
    ],
    'Test Perplexity': [
        # TODO: Fill in
        0.0, 0.0, 0.0, 0.0
    ]
}

df = pd.DataFrame(comparison_data)
print("\nModel Comparison:")
print(df.to_string(index=False))

### Exercise 9.2: Visualize Results

In [ ]:
# TODO: Create visualizations:
# 1. Bar chart of perplexities
# 2. Top-20 most common words
# 3. Top-20 most common bigrams

# Example for most common words:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Most common unigrams
# TODO: YOUR CODE HERE

# Plot 2: Most common bigrams
# TODO: YOUR CODE HERE

plt.tight_layout()
plt.show()

## Part 10: Bonus Challenges

### Challenge 1: Try Different Datasets

Load a different dataset from HuggingFace and train your models on it.

Suggestions:
- `"wikitext"` (different versions)
- `"bookcorpus"`
- `"ptb_text_only"` (Penn Treebank)
- `"imdb"` (movie reviews)

In [ ]:
# TODO: YOUR CODE HERE
# Try loading and training on a different dataset

### Challenge 2: Implement Better Smoothing

Implement add-k smoothing (where k < 1) or interpolation smoothing.

In [ ]:
# TODO: YOUR CODE HERE

### Challenge 3: Build a Simple Autocomplete System

Given a partial sentence, suggest the top-k most likely next words.

In [ ]:
def autocomplete(partial_sentence: str, model: BigramModel, k: int = 5) -> List[Tuple[str, float]]:
    """
    Suggest next words for autocomplete.

    Args:
        partial_sentence: Incomplete sentence
        model: Trained bigram model
        k: Number of suggestions

    Returns:
        List of (word, probability) tuples
    """
    # TODO: Implement this
    pass

# Test
test_partial = "the cat sat on the"
suggestions = autocomplete(test_partial, bigram_model, k=5)
print(f"Suggestions for: '{test_partial}'")
for word, prob in suggestions:
    print(f"  {word}: {prob:.4f}")